# Purpose: We aim to make use of real time streamed data 
- get stream of incoming data on lsl
- collect data for a second
- run inference on the data of that second
- continuously give output, collect new data

In [1]:
# Imports
from pylsl import StreamInlet, resolve_stream
import time
import numpy as np
import torch
import torch.nn as nn

# Initialize and load trained model

In [2]:
# Init NN model
class NeuralNetworkClassificationModel(nn.Module):
    def __init__(self,input_dim,output_dim):
        super(NeuralNetworkClassificationModel,self).__init__()
        self.input_layer    = nn.Linear(input_dim,128)
        self.hidden_layer1  = nn.Linear(128,64)
        self.output_layer   = nn.Linear(64,output_dim)
        self.relu = nn.ReLU()
    
    
    def forward(self,x):
        out =  self.relu(self.input_layer(x))
        out =  self.relu(self.hidden_layer1(out))
        out =  self.output_layer(out)
        return out

# Load model
input_dim  = 16
output_dim = 2
model = NeuralNetworkClassificationModel(input_dim,output_dim)
model.load_state_dict(torch.load("Saved models/toTestIfRealtimeWorks.pth"))

# Define functions for getting observed data, and get inference on data

In [24]:
# get data func
def getData(inlet):
    items = []
    t_end = time.time() + 0.5

    while time.time() < t_end:
        # get a new sample (you can also omit the timestamp part if you're not
        # interested in it)
        sample, timestamp = inlet.pull_sample()
        items.append(sample)
    return items

# get inference func (longest time it took on pracc data was 0.003 sec. that means i can have shorter data collection 
# )
def getInference(samples):
    tensor = torch.tensor(samples)
    predictions = []
    final = []
    with torch.no_grad():
        predictions = model(tensor)

    predictions = predictions.numpy()

    for i in range(len(predictions)):
            final.append(np.argmax(predictions[i]))
    final = np.array(final)    
    print("the most occuring predicted class in the samples:", np.argmax(np.bincount(final)), "with percentage:", np.bincount(final)[np.argmax(np.bincount(final))] / len(final), "out of:", len(final))
    mostOccuringClass = np.argmax(np.bincount(final))

    return final, mostOccuringClass

# Run functions to get data and inference

In [25]:
# first resolve an EEG stream on the lab network
print("looking for an EEG stream...")
streams = resolve_stream('type', 'EEG')

# create a new inlet to read from the stream
inlet = StreamInlet(streams[0])

t_endOuter = time.time() + 10
starterTime = time.time()

while time.time() < t_endOuter:
    samples = getData(inlet)
    predictions = getInference(samples)

finisherTime = time.time()
print("total time taken", finisherTime - starterTime)


looking for an EEG stream...
the most occuring predicted class in the samples: 1 with percentage: 0.9090909090909091 out of: 66
the most occuring predicted class in the samples: 1 with percentage: 0.8769230769230769 out of: 65
the most occuring predicted class in the samples: 1 with percentage: 0.8923076923076924 out of: 65
the most occuring predicted class in the samples: 1 with percentage: 0.8923076923076924 out of: 65
the most occuring predicted class in the samples: 1 with percentage: 0.8769230769230769 out of: 65
the most occuring predicted class in the samples: 1 with percentage: 0.9230769230769231 out of: 65
the most occuring predicted class in the samples: 1 with percentage: 0.8769230769230769 out of: 65
the most occuring predicted class in the samples: 1 with percentage: 0.9142857142857143 out of: 70
the most occuring predicted class in the samples: 1 with percentage: 0.8923076923076924 out of: 65
the most occuring predicted class in the samples: 1 with percentage: 0.892307692